# Elastic search ingester 

## Work Plan: 

- implement project readers in pool_tools
   - every reader returns elastic search ingest items
- implement directory walker, generating ingest reqests
   - ingest request is input to project reader
- use http://python-rq.org/ to queue ingest items
- use parallel workers to run elastic search ingest items
   - each worker maintains an elesticsearch endpoint


In [ ]:
from pindex import pool_tools

## References
- https://janakiev.com/blog/python-filesystem-analysis/

In [10]:
#!pip install persist-queue[extra]
!pip install rq

    100% |████████████████████████████████| 51kB 1.7MB/s 
  Stored in directory: /home/stephan/.cache/pip/wheels/91/76/0f/0b400b897b4d2b889383bc25d791d6116d6c2ab9044a4ba35a
Successfully built rq


In [2]:
from persistqueue import Queue
q = Queue("mypath")
q.put('a')
q.put('b')
q.put('c')
q.get()

Default tempdir '/tmp/tmp3j2i9ke6' is not on the same filesystem with queue path 'mypath',defaulting to 'mypath'.


'a'

In [3]:
q.get()

'b'

In [7]:
import os
dirpaths  = [f.path for f in os.scandir('/run/media/stephan/Volume/data') if f.is_dir()]
dirpaths

['/run/media/stephan/Volume/data/hus',
 '/run/media/stephan/Volume/data/pr',
 '/run/media/stephan/Volume/data/ta',
 '/run/media/stephan/Volume/data/tas',
 '/run/media/stephan/Volume/data/tasmax',
 '/run/media/stephan/Volume/data/tasmin']

In [10]:
import os

proj = "cmip6"
sh = pool_tools.SHelper(proj=proj, start_dir="/run/media/stephan/Volume/data")
print("SHelper initialized:", sh.start_dir)
es = sh.get_es()
print(es)
walk = os.walk(sh.start_dir)
i = 0

for root, dirs, files in walk:
    if len(files) > 0: 
        print("=============================: ",root,dirs,files)
        res = sh.worker(root, dirs, files)
        # update elastic search
        i_list = []
        for item, i_dict in res.items():
             i += 1
             print("index update: ", i)
             if len(i_dict) > 0:
                index_res = es.index(index=proj, id=i, body=i_dict)
        print("============================")


SHelper initialized: /run/media/stephan/Volume/data
Elastic search endpoint:  [{'host': 'localhost', 'port': 9200}]
<Elasticsearch([{'host': 'localhost', 'port': 9200}])>
=============================:  /run/media/stephan/Volume/data/hus ['gr1'] ['hus_day_IPSL-CM6A-LR_abrupt-4xCO2_r10i1p1f1_gr_18501001-18541231.nc']
index update:  1
=============================:  /run/media/stephan/Volume/data/hus/gr1/v20180701 [] ['hus_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_015101-025012.nc', 'hus_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_025101-035012.nc', 'hus_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_035101-045012.nc', 'hus_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_045101-055012.nc', 'hus_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_055101-065012.nc']
index update:  2
index update:  3
index update:  4
index update:  5
index update:  6
=============================:  /run/media/stephan/Volume/data/pr [] ['pr_Amon_GFDL-AM4_amip_r1i1p1f1_gr1_198001-201412.nc', 'pr_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_015101-025012.nc', 'pr_A

In [ ]:
## bulk update
es = Elasticsearch()
>>> # k is a generator expression that produces
... # a series of dictionaries containing test data.
... # The test data are just letter permutations
... # created with itertools.permutations.
... #
... # We then reference k as the iterator that's
... # consumed by the elasticsearch.helpers.bulk method.
>>> k = ({'_type':'foo', '_index':'test','letters':''.join(letters)}
...      for letters in itertools.permutations(string.letters,2))

>>> # calling k.next() shows examples
... # (while consuming the generator, of course)
>>> # each dict contains a doc type, index, and data (at minimum)
>>> k.next()
{'_type': 'foo', 'letters': 'ab', '_index': 'test'}
>>> k.next()
{'_type': 'foo', 'letters': 'ac', '_index': 'test'}
>>> # create our test index
>>> es.indices.create('test')
{u'acknowledged': True}
>>> helpers.bulk(es,k)
(2650, [])
>>> # check to make sure we got what we expected...
>>> es.count(index='test')

In [4]:
q_string = {"query": {
               "bool": {
                   "must": [
                       {"match": {"variable": "tas"}},
                       {"match": {"model":"GFDL-CM4"}},
                       {"match": {"time":'015101-025012'}}
                                  ]             
                      }
               }
           }
r_string = {"query": {
               "bool": {
                   "must": [
                       {"match": {"stime": "055101"}},
                       {"match": {"model":"GFDL-CM4"}}                   
                                  ]             
                      }
               }
           }

r_string = {"query": {
               "bool": {
                   "filter": [
                       {"term": {"stime":"055101"}}
                   ]
                      }
               }
           }

t_string = {"query": {
              "filter" : {
                 "term" : { "stime": "015101"}
                }
            }}

In [5]:
import pprint
pp=pprint.PrettyPrinter(indent=4)
res = es.search(index="cmip6", body=r_string)
pp.pprint(res['hits']['hits'])

[   {   '_id': '6',
        '_index': 'cmip6',
        '_score': 0.0,
        '_source': {   'dataset_id': '/run/media/stephan/Volume/data/hus/gr1/v20180701',
                       'etime': '065012',
                       'experiment': 'piControl',
                       'file_name': 'hus_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_055101-065012.nc',
                       'grid': 'gr1',
                       'member': 'r1i1p1f1',
                       'model': 'GFDL-CM4',
                       'project': 'cmip6',
                       'stime': '055101',
                       'table': 'Amon',
                       'time': '055101-065012',
                       'variable': 'hus'},
        '_type': '_doc'},
    {   '_id': '21',
        '_index': 'cmip6',
        '_score': 0.0,
        '_source': {   'dataset_id': '/run/media/stephan/Volume/data/ta/gr1/v20180701',
                       'etime': '065012',
                       'experiment': 'piControl',
                       'file_nam

In [7]:
import redis

r = redis.Redis(password='prolog1..')


In [8]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


In [9]:
r.save()

True